# Another try at Figure 2

### Imports

In [ ]:
import numpy as np

### Generate Data

In [ ]:
def get_multivariate_sample(n, d, mean):
    x_sample = []
    y_sample = []
    means = np.zeros(d)
    means[0] = mean
    for i in range(n):
        y = np.random.binomial(1, .5)
        if (y == 0):
            x = np.random.multivariate_normal(-means, np.identity(d))
        else:
            x = np.random.multivariate_normal(means, np.identity(d))
        x_sample.append(x.tolist())
        y_sample.append(y)
    return np.asarray(x_sample), np.asarray(y_sample)

### Richard's Original CEF

In [ ]:
# This is Richard's original CEF function.  Don't mess with it.
from sklearn.ensemble.forest import _generate_unsampled_indices
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
import math

def cef_estimate_richard(X, y, n_estimators = 200, max_samples = .32, bootstrap = True, depth = 30, min_samples_leaf = 1, max_features = 1.):
    model = BaggingClassifier(DecisionTreeClassifier(max_depth = depth, min_samples_leaf = min_samples_leaf, max_features = math.ceil(int(math.sqrt(X.shape[1])))), 
                              n_estimators = n_estimators, 
                              max_samples= max_samples, 
                              bootstrap = bootstrap)
    model.fit(X, y)
    class_counts = np.zeros((X.shape[0], model.n_classes_))
    for tree in model:
        
        # get out of bag indices.       
        # Here's where we obtain unsampled indices.
        unsampled_indices = _generate_unsampled_indices(tree.random_state, len(X), int((1 - max_samples)*len(X)))
        # Done with unsampled indices.
        
        total_unsampled = len(unsampled_indices)
        np.random.shuffle(unsampled_indices)
        prob_indices, eval_indices = unsampled_indices[:total_unsampled//2], unsampled_indices[total_unsampled//2:]
        # get all node counts
        node_counts = tree.tree_.n_node_samples
        # get probs for eval samples
        posterior_class_counts = np.zeros((len(node_counts), model.n_classes_))
        for prob_index in prob_indices:
            posterior_class_counts[tree.apply(X[prob_index].reshape(1, -1)).item(), y[prob_index]] += 1
        row_sums = posterior_class_counts.sum(axis=1)
        row_sums[row_sums == 0] = 1
        class_probs = (posterior_class_counts/row_sums[:, None])
        
        where_0 = np.argwhere(class_probs == 0)
        for elem in where_0:
            class_probs[elem[0], elem[1]] = 1/(2*row_sums[elem[0], None])
        where_1 = np.argwhere(class_probs == 1)
        for elem in where_1:
            class_probs[elem[0], elem[1]] = 1 - 1/(2*row_sums[elem[0], None])
        
        class_probs.tolist()
        partition_counts = np.asarray([node_counts[x] for x in tree.apply(X[eval_indices])])
        # get probability for out of bag samples
        eval_class_probs = [class_probs[x] for x in tree.apply(X[eval_indices])]
        eval_class_probs = np.array(eval_class_probs)
        # find total elements for out of bag samples
        elems = np.multiply(eval_class_probs, partition_counts[:, np.newaxis])
        # store counts for each x (repeat fhis for each tree)
        class_counts[eval_indices] += elems
    # calculate p(y|X = x) for all x's
    probs = class_counts/class_counts.sum(axis = 1, keepdims = True)
    entropies = -np.sum(np.log(probs)*probs, axis = 1)
    # convert nan to 0
    entropies = np.nan_to_num(entropies)
    return np.mean(entropies)

### Mike's Altered CEF 1

In [ ]:
# All we're changing is how we get unsampled indices.  This avoids using the deprecated
# _generate_unsampled_indices function, which we know yields some sampled indices.
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
import math

def cef_estimate_mike_1(X, y, n_estimators = 200, max_samples = .32, bootstrap = True, depth = 30, min_samples_leaf = 1, max_features = 1.):
    model = BaggingClassifier(DecisionTreeClassifier(max_depth = depth, min_samples_leaf = min_samples_leaf, max_features = math.ceil(int(math.sqrt(X.shape[1])))), 
                              n_estimators = n_estimators, 
                              max_samples= max_samples, 
                              bootstrap = bootstrap)
    model.fit(X, y)
    class_counts = np.zeros((X.shape[0], model.n_classes_))
    tree_idx = 0
    for tree in model:
        
        # get out of bag indices.       
        # Here's where we obtain unsampled indices.
        # unsampled_indices = _generate_unsampled_indices(tree.random_state, len(X), int((1 - max_samples)*len(X)))
        sampled_indices = model.estimators_samples_[tree_idx]
        unsampled_indices = np.delete(np.arange(0,X.shape[0]), sampled_indices)
        tree_idx = tree_idx + 1
        # Done with unsampled indices.
        
        total_unsampled = len(unsampled_indices)
        np.random.shuffle(unsampled_indices)
        prob_indices, eval_indices = unsampled_indices[:total_unsampled//2], unsampled_indices[total_unsampled//2:]
        # get all node counts
        node_counts = tree.tree_.n_node_samples
        # get probs for eval samples
        posterior_class_counts = np.zeros((len(node_counts), model.n_classes_))
        for prob_index in prob_indices:
            posterior_class_counts[tree.apply(X[prob_index].reshape(1, -1)).item(), y[prob_index]] += 1
        row_sums = posterior_class_counts.sum(axis=1)
        row_sums[row_sums == 0] = 1
        class_probs = (posterior_class_counts/row_sums[:, None])
        
        where_0 = np.argwhere(class_probs == 0)
        for elem in where_0:
            class_probs[elem[0], elem[1]] = 1/(2*row_sums[elem[0], None])
        where_1 = np.argwhere(class_probs == 1)
        for elem in where_1:
            class_probs[elem[0], elem[1]] = 1 - 1/(2*row_sums[elem[0], None])
        
        class_probs.tolist()
        partition_counts = np.asarray([node_counts[x] for x in tree.apply(X[eval_indices])])
        # get probability for out of bag samples
        eval_class_probs = [class_probs[x] for x in tree.apply(X[eval_indices])]
        eval_class_probs = np.array(eval_class_probs)
        # find total elements for out of bag samples
        elems = np.multiply(eval_class_probs, partition_counts[:, np.newaxis])
        # store counts for each x (repeat fhis for each tree)
        class_counts[eval_indices] += elems
    # calculate p(y|X = x) for all x's
    probs = class_counts/class_counts.sum(axis = 1, keepdims = True)
    entropies = -np.sum(np.log(probs)*probs, axis = 1)
    # convert nan to 0
    entropies = np.nan_to_num(entropies)
    return np.mean(entropies)

### Experiments

### d = 1, mu = 1

In [ ]:
from tqdm import tqdm_notebook

cef_r = []
cef_m1 = []

d = 1
mean = 1

sample_sizes = [1000, 2000, 4000, 8000, 16000, 32000, 64000, 96000]
traces = 5

for trace in range(traces):
    print(trace)
    np.random.seed(123*trace)
    X_full, y_full = get_multivariate_sample(np.max(sample_sizes), d, mean)
    for elem in tqdm_notebook(sample_sizes):
        X = X_full[0:elem, :]
        y = y_full[0:elem]
        cef_r.append(cef_estimate_richard(np.array(X), y, 300, .32, depth = 30))
        cef_m1.append(cef_estimate_mike_1(np.array(X), y, 300, .32, depth = 30))

In [ ]:
import pickle 
file = open('figure_2_cef_r_d1_mean1_20200127.pkl', 'wb') 
pickle.dump(cef_r, file)
file = open('figure_2_cef_m1_d1_mean1_20200127.pkl', 'wb') 
pickle.dump(cef_m1, file)

In [ ]:
cef_r_traces = np.asarray(cef_r).reshape((traces,len(sample_sizes))).T
cef_m1_traces = np.asarray(cef_m1).reshape((traces,len(sample_sizes))).T

In [ ]:
cef_r_mean = np.mean(cef_r_traces, axis = 1)
cef_m1_mean = np.mean(cef_m1_traces, axis = 1)

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

sns.set(font_scale = 2.5)
sns.set_style("ticks")
plt.figure(figsize=(12, 6))
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams['figure.figsize'] = [15, 10]

for trace in range(traces):
    line_cef_r, = plt.plot(sample_sizes, cef_r_traces[:, trace], label = "CEF_R", linewidth = .5, linestyle = "solid", color = "red")
    line_cef_m1, = plt.plot(sample_sizes, cef_m1_traces[:, trace], label = "CEF_M1", linewidth = .5, linestyle = "dashdot", color = "blue")
line_true = plt.axhline(y=.356, linestyle='-', color = "black", label = "Truth")
plt.plot(sample_sizes, cef_r_mean, linewidth = 2, linestyle = "solid", color = "red")
plt.plot(sample_sizes, cef_m1_mean, linewidth = 2, linestyle = "solid", color = "blue")

plt.ylim(ymin = 0)
plt.ylim(ymax = .8)
plt.legend(handles=[line_cef_r, line_cef_m1, line_true])
plt.xlabel('n')
plt.ylabel("$\hat H(Y|X)$")
plt.tight_layout()
plt.title("Conditional entropy estimate vs \nincreasing n (d = 1, effect size = 1)")
plt.savefig("alg_comp_1d_mean1_20200127.png")
plt.show()

### d = 40, mu = 1

In [ ]:
from tqdm import tqdm_notebook

cef_r = []
cef_m1 = []

d = 40
mean = 1

sample_sizes = [i*10000 for i in np.arange(1, 11)]
traces = 5

for trace in range(traces):
    print(trace)
    np.random.seed(123*trace)
    X_full, y_full = get_multivariate_sample(np.max(sample_sizes), d, mean)
    for elem in tqdm_notebook(sample_sizes):
        X = X_full[0:elem, :]
        y = y_full[0:elem]
        cef_r.append(cef_estimate_richard(np.array(X), y, 300, .32, depth = 30))
        cef_m1.append(cef_estimate_mike_1(np.array(X), y, 300, .32, depth = 30))

In [ ]:
import pickle 
file = open('figure_2_cef_r_d40_mean1_20200127.pkl', 'wb') 
pickle.dump(cef_r, file)
file = open('figure_2_cef_m1_d40_mean1_20200127.pkl', 'wb') 
pickle.dump(cef_m1, file)

In [ ]:
cef_r_traces = np.asarray(cef_r).reshape((traces,len(sample_sizes))).T
cef_m1_traces = np.asarray(cef_m1).reshape((traces,len(sample_sizes))).T

In [ ]:
cef_r_mean = np.mean(cef_r_traces, axis = 1)
cef_m1_mean = np.mean(cef_m1_traces, axis = 1)

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline

sns.set(font_scale = 2.5)
sns.set_style("ticks")
plt.figure(figsize=(12, 6))
plt.rcParams["font.family"] = "sans-serif"
plt.rcParams['figure.figsize'] = [15, 10]

for trace in range(traces):
    line_cef_r, = plt.plot(sample_sizes, cef_r_traces[:, trace], label = "CEF_R", linewidth = .5, linestyle = "solid", color = "red", alpha = .5)
    line_cef_m1, = plt.plot(sample_sizes, cef_m1_traces[:, trace], label = "CEF_M1", linewidth = .5, linestyle = "solid", color = "blue", alpha = .5)
line_true = plt.axhline(y=.356, linestyle='-', color = "black", label = "Truth")
plt.plot(sample_sizes, cef_r_mean, linewidth = 2, linestyle = "solid", color = "red")
plt.plot(sample_sizes, cef_m1_mean, linewidth = 2, linestyle = "solid", color = "blue")

plt.ylim(ymin = 0)
plt.ylim(ymax = .8)
plt.legend(handles=[line_cef_r, line_cef_m1, line_true])
plt.xlabel('n')
plt.ylabel("$\hat H(Y|X)$")
plt.tight_layout()
plt.title("Conditional entropy estimate vs \nincreasing n (d = 40, effect size = 1)")
plt.savefig("alg_comp_40d_mean1_20200127.png")
plt.show()